In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
titanic = pd.read_csv('titanic_dataset.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Preprocessing

In [3]:
#Finding null values in the dataset
titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

There are 177, 687 and 2 null values in Age, Cabin and Embarked columns respectively

In [4]:
titanic.shape

(891, 12)

In [5]:
titanic.drop(['Cabin'],axis=1,inplace=True)

In [6]:
titanic['Age']=titanic['Age'].fillna(titanic['Age'].mean())

In [7]:
#as Embarked column has only two null values we can use mode to fill that
titanic['Embarked']=titanic['Embarked'].fillna(titanic['Embarked'].mode()[0])

In [8]:
titanic.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [9]:
titanic.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [10]:
titan=titanic.copy()
titan.drop(['PassengerId','Name','Ticket',],axis=1,inplace=True)

In [11]:
#changing categorical columns to numerical
titan['Sex']=titan['Sex'].replace(['male','female'],[0,1])
titan['Embarked']=titan['Embarked'].replace(['S','C','Q'],[0,1,2])

# StandardScaling

In [12]:
#only Age is to be scaled
from sklearn import preprocessing
standardise=preprocessing.StandardScaler()
titanic[['Age']]=standardise.fit_transform(titanic[['Age']])

# Naive Bayes Classification

In [13]:
#splitting into dependent and independent variables
X=titan.drop(['Survived'],axis=1)
y=titan['Survived']

In [14]:
#now splitting into test and train dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.20)

In [16]:
from sklearn.naive_bayes import GaussianNB
NB_model=GaussianNB()
NB_model.fit(X_train,y_train)
y_pred=NB_model.predict(X_test)

In [17]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print("Accuracy is:",round(accuracy_score(y_test,y_pred),3))

[[85 20]
 [21 53]]
Accuracy is: 0.771


# Hyperparameter tuning

In [19]:
import numpy as np
from sklearn.model_selection import GridSearchCV
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
NB_grid = GridSearchCV(estimator=GaussianNB(), 
                 param_grid=params_NB, 
                 cv=10,   
                 verbose=1, 
                 scoring='accuracy') 
NB_grid.fit(X_train, y_train)
print(NB_grid.best_params_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'var_smoothing': 1.873817422860383e-05}


In [20]:
y_pred=NB_grid.best_estimator_.predict(X_test)

In [21]:
print(confusion_matrix(y_test,y_pred))
print("Accuracy is:",accuracy_score(y_test,y_pred))

[[88 17]
 [21 53]]
Accuracy is: 0.7877094972067039


Accuracy has been improved slightly after Hypertuning